In [1]:
import xarray as xr
import datetime
import os
from shapely.geometry import Point
import gdal
import geopandas
from geopandas.tools import sjoin
import rasterio
import numpy as np
from affine import Affine
from pyproj import Proj, transform
import pandas as pd
import affine
from geopandas import GeoDataFrame

coords = pd.read_csv('supplier_20180806.csv')

coords = coords.rename(columns={'LONGITUDE': 'longitude', 'LATITUDE': 'latitude'})

def RetrieveRasterAppAttr(global_coords, rasterfile, newattr):
    # retrieve pixel values for lat, lon
    t1 = datetime.datetime.now()
    with rasterio.open(rasterfile) as src:
        global_coords[newattr] = [x[0] for x in src.sample(zip(global_coords.longitude, global_coords.latitude))]
    t2 = datetime.datetime.now()
    print('It took ', t2-t1)

C:\Users\hpuzzang\Documents\Python\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
geom = coords.apply(lambda x : Point([x['longitude'],x['latitude']]), axis=1)
coords = geopandas.GeoDataFrame(coords, geometry=geom) #geom is a Series
coords.crs = {'init' :'epsg:4326'}
shapefile=r'\\pngssvmh01\hpuzzang\Geo Data\global_urban_extent_polygons_v1.01.shp'
poly = geopandas.GeoDataFrame.from_file(shapefile)
coords = sjoin(coords, poly.loc[:, ['SCHNM', 'geometry']], how='left').drop(['index_right'], axis = 1)
shapefile=r'\\pngssvmh01\hpuzzang\Geo Data\TM_WORLD_BORDERS_SIMPL-0.3.shp'
poly = geopandas.GeoDataFrame.from_file(shapefile)
coords = sjoin(coords, poly, how='left').drop(['geometry', 'index_right'], axis = 1)
coords['country'] = coords['country'].fillna('')

In [4]:
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_population_density_adjusted_to_2015_unwpp_country_totals_rev10_2015_2pt5_min.tif', 'GPW_pop_2015')
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a000_014bt_2010_cntm_30_sec.tif', 'age0to14cnt')
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a000_014ft_2010_cntm_30_sec.tif', 'age0to14femalecnt')
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a000_014mt_2010_cntm_30_sec.tif', 'age0to14malecnt')
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a015_049bt_2010_cntm_30_sec.tif', 'age15to49cnt')
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a015_049ft_2010_cntm_30_sec.tif', 'age15to49femalecnt')

It took  0:00:09.873013
It took  0:00:42.639978
It took  0:00:42.705000
It took  0:00:41.539000
It took  0:00:36.630000
It took  0:00:38.066000


In [5]:
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a015_049mt_2010_cntm_30_sec.tif', 'age15to49malecnt')
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a015_064bt_2010_cntm_30_sec.tif', 'age15to64cnt')
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a015_064ft_2010_cntm_30_sec.tif', 'age15to64femalecnt')
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a015_064mt_2010_cntm_30_sec.tif', 'age15to64malecnt')

It took  0:00:42.978000
It took  0:00:44.426000
It took  0:00:44.056000
It took  0:00:43.446000


In [13]:
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a065plusbt_2010_cntm_30_sec.tif', 'age65pluscnt')
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a065plusft_2010_cntm_30_sec.tif', 'age65plusfemalecnt')
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_basic_demographic_characteristics_rev10_a065plusmt_2010_cntm_30_sec.tif', 'age65plusmalecnt')

It took  0:00:25.357899
It took  0:00:25.651130
It took  0:00:26.533305


In [14]:
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_land_water_area_rev10_landareakm_30_sec.tif', 'landareakm')
RetrieveRasterAppAttr(coords, r'H:\Geo Data\gpw_v4_land_water_area_rev10_waterareakm_30_sec.tif', 'waterareakm')

RetrieveRasterAppAttr(coords, r'H:\Geo Data\final_trr.tif', 'epopTerro')

It took  0:00:12.741548
It took  0:00:08.043609
It took  0:00:10.176035


In [15]:
# replace the missing values
coords._get_numeric_data()[coords._get_numeric_data() < -100000] = np.NaN
gcoords = coords.fillna(0)

In [16]:
# create smart variables: ratio
# water ratio
gcoords.eval("ratioWater = (waterareakm + 0.000001)/(waterareakm + landareakm + 0.000001)", inplace=True)

In [17]:
# female ratio
gcoords.eval("ratioFemale0to14 = (age0to14femalecnt + 0.000001)/(age0to14cnt + 0.000001)", inplace=True)
gcoords.eval("ratioFemale15to49 = (age15to49femalecnt + 0.000001)/(age15to49cnt + 0.000001)", inplace=True)
gcoords.eval("ratioFemale15to64 = (age15to64femalecnt + 0.000001)/(age15to64cnt + 0.000001)", inplace=True)
gcoords.eval("ratioFemale65plus = (age65plusfemalecnt + 0.000001)/(age65pluscnt + 0.000001)", inplace=True)

In [18]:
# kids ratio
gcoords.eval("ratioKid = (age0to14cnt + 0.000001)/(age0to14cnt + age15to64cnt + age65pluscnt + 0.000001)", inplace=True)

# senior ratio
gcoords.eval("ratioSenior = (age65pluscnt + 0.000001)/(age0to14cnt + age15to64cnt + age65pluscnt + 0.000001)", inplace=True)

In [19]:
gcoords.to_csv(r'latlong_country_tns_id_aig_appended2.csv', sep=',')